# Ancient DNA and the dispersal of domestic cats into Europe

In this hands-on session we will analyse the data from the study of [**De Martino et al.**](https://www.biorxiv.org/content/10.1101/2025.03.28.645893v1.full), in which 70 low-coverage (∼0.07-fold to ∼1.39-fold) ancient genomes of cats dated to the last 10 thousand years were analysed to investigate the introduction of domestic cats to Europe and gene flow across time between wild and domestic cats. 

## 1. Full mtDNA phylogeny

We will start by making a phylogenetic tree of high coverage complete mtDNAs. To do that we will work in bash, and more particularly we will use a series of tools that were installed in a conda environment called *bioinf*.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> Make sure that the notebook's Kernel (top-right of this notebook) is set to <b>bioinf</b>, so as to activate the conda environment and work in bash. 
</div>

To make sure all the programs installed in the conda environment are found, we need to add the path of the conda environment’s *bin* folder to the PATH variable in the notebook. <span style="color:red">**Please note that the path must be changed based on the location where you are working in your local machine.**<span>

In [ ]:
import os
os.environ["PATH"] = "/Users/claudio/miniconda3/envs/bioinf/bin:" + os.environ["PATH"]
#os.environ["PATH"] = "/home/studente/miniconda3/envs/bioinf/bin:" + os.environ["PATH"]

You can check the content of the folder in which you are working by typing the bash command *ls*.

In [ ]:
ls

The current working folder contains the *fasta* multiple alignment file of all the mtDNAs generated in the study by De Martino et al., plus additional samples from the literature. To run some bash commands in Jupyter you will have to type an exclamation mark (!) before the command itself (e.g. *head*, *less*). 

We will now run a Maximum-likelihood tree of the mtDNA sequences in the fasta aligment with the program [**IQ-TREE**](https://iqtree.github.io/doc/).

In [ ]:
# Run iqtree
!iqtree -s Cats_alignment_mtDNA_v8_withOUTGROUP.fasta -m MFP -o FMA08 -redo

<div class="alert alert-block alert-warning">
<b>NOTA:</b> Impostare la Kernel del Notebook sull'environment conda <b>ete3<b>. 
</div>

Let's explore now the output of the IQ-TREE run. The program generated the tree in a file with the extension *.treefile*. We can draw the ML-Tree with a python tool called [**ETE-Toolkit**](https://etetoolkit.org/docs/latest/tutorial/index.html).

In [ ]:
# import with python the ete3 tools and store the tree in an object called t. 
from ete3 import (Tree, PhyloTree)
t = Tree("Cats_alignment_mtDNA_v8_withOUTGROUP.fasta.treefile")
# let's make clear in the tree that the outgroup we are using is the samples FMA08 (Felis margarita)
t.set_outgroup("FMA08")

In [ ]:
# to plot the tree
t.render("%%inline")

More in general, you can import the *treefile* in [**FigTree**](https://tree.bio.ed.ac.uk/software/figtree/) to have a closer look at the tree and customize it with colors for different clades.

To slightyl edit and customize the tree here, we can run the following command amd highlight some samples, in particular the ancient samples from Anatolia (ASK01, BA02, BA03, MENT01) and Bulgaria (DUR01, DUR02, DUR03, DUR04) date to the Neolithic and the Chalcolithic. 

In [ ]:
# customize symbol size and color
from ete3 import Tree, TreeStyle, NodeStyle, faces, AttrFace, TextFace
symbol_map = {
    "ASK01_merged_mtDNA_final.": ("circle", "red"),
    "BA02_DpS_AE_final.": ("circle", "red"),
    "BA03_merged_mtDNA_final.": ("circle", "red"),
    "MENT01_DpSAN_final.": ("circle", "red"),
    "DUR01_DpS_AE_final.": ("circle", "blue"),
    "DUR02_DpS_AE_final.": ("circle", "blue"),
    "DUR03_DpS_AD_final.": ("circle", "blue"),
    "KOP04_merged_mtDNA_final.": ("circle", "blue"),
}

for leaf in t.iter_leaves():
    shape, color = symbol_map.get(leaf.name, ("circle", "black"))
    nstyle = NodeStyle()
    nstyle["shape"] = shape
    nstyle["size"] = 8
    nstyle["fgcolor"] = color
    leaf.set_style(nstyle)

In [ ]:
# to plot the tree
t.render("%%inline")

You can compare the output with the original figure of the study, where the various clades are better identified. 
<span style="color:red">**Where do the samples from Neolithic Anatolia (samples ASK01, BA02, BA03, MENT01) cluster?**</span>

<center><img src="images/Fig_mtDNA_tree.pdf"
            width="500"/></center>

## 2. PCA of genome-wide data

Let's now have a look at the genome data. A panel of single nucleotide polymorphisms (SNPs) was built by calling variants in present-day wild and domestic cats generated in  De Martino et al. and from the literature (n=56, coverage >10-fold). These SNPs were used to call pseudohaploid genotypes in both ancient and low- to mid-coverage modern genomes (coverage <10-fold). Only autosomal variants were retained in all the panels. For the downstream population genomic analyses including ancient samples, transitions were excluded in order to exclude any potential bias from *post-mortem* damage (miscoding lesions C-T and G-A). 

You can find below a Principal Component Analysis (PCA) built with *smartpca*, which is part of the [**EIGENSOFT**](https://github.com/DReichLab/EIG) package. With *smartpca*, low coverage samples are projected onto the coordinate space defined by modern mid- to high-range coverage wild and domestic cats.

<center><img src="images/PCA.png"
            width="900"/></center>

The PCA is showing that that Neolithic and Chalcolithic cats from Anatolia and Bulgaria (blue and red circles) which possessed a mtDNA typical of *Felis lybica lybica* are actually European wildcats (*F. silvestris*)! Conflicting mitochondrial and nuclear evolutionary histories (i.e. **mitonuclear discordance**) may be due either to hybridization or incomplete lineage sorting (ILS). In this case, given the clear biogeographic pattern of mitonuclear discordance as illustrated in the ancient European wildcats (it is present in Anatolia and Southeast Europe, but absent in Central and Western Europe), ILS can be ruled out. You can read more about this in the manuscripts by [Toews et al. 2012](https://onlinelibrary.wiley.com/doi/10.1111/j.1365-294X.2012.05664.x) and [Funk and Omeland 2003](https://www.annualreviews.org/content/journals/10.1146/annurev.ecolsys.34.011802.132421).


## 3. Admixtools - preparation of the dataset
To investigate genetic relationships among the samples of the dataset and the patterns of hybridization between wild and domestic cats across time we will use [**ADMIXTOOLS 2**](https://uqrmaie1.github.io/admixtools/index.html) and the [***f*- and *D*-statistics**](https://uqrmaie1.github.io/admixtools/articles/fstats.html). 

<div class="alert alert-block alert-warning">
<b>NOTE:</b> Make sure that the notebook's Kernel is set on R.
</div>

#### 3.1. Preparation of the dataset

First, we have to set the path to the folder containing the files that we will use in this practical session. We are saving the path in a variable called "dd", that will be used later by the R package [**glue**](https://www.rdocumentation.org/packages/glue/versions/1.8.0) as shortcut of the path. <span style="color:red">**Please note that the path must be changed based on the location where you are working in your local machine.**<span>

In [ ]:
# this path must be changed based on the position of the working folder in your local machine.
dd = '/Users/claudio/Documents/WORK/4-BIOINFORMATICS_OTHER/Jupyter_notebooks/cat_paleogenomics_Apr2025'
#dd = '/home/studente/cat_paleogenomics_Apr2025'

Then, we activate the R libraries needed for the analysis. Pay attention to any error messages returned while loading the libraries. The missing libraries must be installed directly in R. 

In [ ]:
library(admixtools)
library(genio)
library(glue)
library(gplots)
library(tidyverse)
library(data.table)
options(warn=-1) #switch off warnings 
#options(warn=0) # switch on warnings

We will use, amongst others, R functions prepared by [Benjamin Peter](https://github.com/BenjaminPeter) to create charts. To activate these functions in R, we must call a file in which they are contained with the command *source*. Below, we use *glue* to paste the path containd in dd to the name of the file (*analysis.R*), which is contained in the working folder.

In [ ]:
source(glue('{dd}/analysis.R'))

#### 3.2. Importing the panel input files
To analysed the genome-wide data in ancient and modern individuals we will use specific file formats, called [**eigenstrat**](https://reich.hms.harvard.edu/software/InputFileFormats), to store information about SNPs at the genome wide level from individuals originating from different populations. All the information (SNPs positions for each individual, population of origin and other possible metadata) are stored in three files, *.snp*, *.ind*, *.geno*. Here is a schematization of how their layout. 

<center><img src="images/eigenstrat.png"
            width="900"/></center>

Let's first import the *file.ind* containing the information (metadata) of the individual analysed. We are importing the data as a table with three columns, corresponding to the ID of the inidividual, sex (this can be omitted) and the population to which the individual belong. 

In [ ]:
ind_all <- readr::read_table(glue("{dd}/panel_cats_2025_Europe/final_merged_panel_ind131_maf002_LD50502_Tv.ind"), col_names=c("ind", "sex", "pop"), col_types='ccc')


We can view the entire list of individuals by typing *ind_all*, or even just the list of populations present in the dataset we are analyzing by typing *ind_all$pop*. We could also create a list of all the populations sorted by name to consult upon neccessity. 

In [ ]:
pops = sort(unique(ind_all$pop))
ind_all

Next, we can create three new objects. The first returns the path to the folder containing the SNPs dataset we are analyzing (*filename.geno*, *filename.ind*, *filename.snp*). In order for admixtools to be able to read these three files we need to add the name (without the final extension) of the three files, which must be the same for all three, to the path of the folder containing them. The *f2_all* object is the path to the subfolder (which was created specifically beforehand) inside which the results of admixtools will be saved.

In [ ]:
panel = glue('{dd}/panel_cats_2025_Europe/final_merged_panel_ind131_maf002_LD50502_Tv')
f2_all = glue('{dd}/panel_cats_2025_Europe/f2_values')

#### 3.3. Reading the panel and estimating the *f*2 values
We first estimate the values of *f*2, which will then be used to calculate the *f*3 and *f*4 as well. The following command allows admixtools to read the SNPs data for each individual composing our dataset. 

In [ ]:
geno_ancient = read_eigenstrat(panel)

<span style="color:red">**How many SNPs have been read?**</span>

After importing the data, we estimate the *f*2 values. The *blgsize* option indicates the length of the chromosome segments (in centimorgans) to be used as ‘blocks’ for the [**jackknife**](https://uqrmaie1.github.io/admixtools/articles/resampling.html) analysis,  which allows us to calculate the standard errors of our estimates (click on the link to learn more).
The calculation takes time, for this reason we have already calculated the *f*2 values for the entire dataset (modern and ancient samples), saving the results in the appropriate folder (*f2_values*). Since there are many ancient genomes of low quality (coverage <1), we use the `maxmiss=1` option, which allows us to maximize the number of SNPs for each pair of samples rather than working with the same set of SNPs for all comparisons (which is possible with high-quality genomes, such as modern ones). Just for your knowledge, the following command is used to estimate the *f*2 values (the command is currenly commented to avoid running it accidentally). 

In [ ]:
#admixtools::extract_f2(panel, f2_all, blgsize = 0.05, overwrite=TRUE, maxmiss = 1)

## 4. *f*3-outgroup analysis
Once estimated the *f*2 values, we can use them to calculate the *f*3 values. These can be used to infer the genetic distance between two populations by calculating the outer branch from an **outgroup** (i.e., the drift accumulated from the node separating them to the outgroup, see the red arrow in the figure below). In this configuration, the test is called *f*3-outgroup and we assume that the outgroup didn't mix with the two populations under consideration. 


<center><img src="images/F3_outgroup_figure.png"
            width="300"/></center>

We can now create different vectors corresponding to categories of samples (e.g. geographic regions, domestic cats etc.) that we can use  in the following analyses. 
Remember that you can always use the vector *pops* that we generated later to visualize the samples. 

In [ ]:
wc_italy = c('1294_Fss','1302_Fss','1611_Fss','66_Fss','78_Fss','79_Fss','99_Fss','997_Fss')
wc_lybica = c('778_Fsl','779_Fsl','FSI47_Fsl','FSI48_Fsl','FSI51_Fsl')
wc_levant = c('FSI47_Fsl','FSI48_Fsl','FSI51_Fsl')
wc_morocco = '778_Fsl'
wc_tunisia = '779_Fsl'
dom_modern = c('CGAE18351_DevonRex','CGAE2613_Sphynx','CGAE6166_MaineCoon','CGAE6197_DomesticLonghair','DS05501_RandomBred','Fcat16274_Ragdoll',
               'Fcat18666_RandomBred','Fcat18801_RandomBred','Fcat20680_EgyptianMau','Fcat20784_AmericanWirehair','Fcat20822_Ocicat','Fcat20907_SelkirkRex',
               'Fcat22054_Siberian','Fcat22335_TurkishAngora','Fcat22361_Persian','Hills2_RandomBred','Katze169_RandomBred','Sizzle_DomesticShorthair')
dom_ancient = c('BHHL01','BHHL03','BP01','BP02','BP03','BREM05','BREM07','BRPA24','BRPA27','CAST02',
                'CEC03','DH06','GRA01','GSA01','HAIS01','HAIS04','HAIS10','IRT01','LAB01','LHM02',
                'LHM03','MRY07','MRY08','MRY09','MRY10','MTR01B','MUS01','PAD01','PAL09','PAL10',
                'PNE02','PNE03','SA14','SBR01','SCAT01','SDUC02B')
wc_sardinia = c('46_Fsl','52_Fsl','53_Fsl','55_Fsl')

We can now compute the *f*3 values between modern domestic cats and African wildcats. To do this, we will use the *f*2 values previously estimated on the entire dataset (*f2_all*). The Jungle cat (*Felis chaus*) is used as an outgroup. The populations will be analyzed in rotation using the vectors created earlier. We will also print the table with the results and plot them in separate charts. 

In [ ]:
f3_out = f3(f2_all, pop1='Felis_chaus', pop2=wc_lybica, pop3=dom_modern) %>% arrange(est)

In [ ]:
library(ggplot2)

In [ ]:
# Convert the f3 values (est) and the standard error (se) to numeric
f3_out <- f3_out %>%
  mutate(
    f3 = as.numeric(est),
    stderr = as.numeric(se)
  )

In [ ]:
# set the options for the size of the plot
library(repr)
options(repr.plot.width=16, repr.plot.height=16)

In [ ]:
# plot the results
ggplot(f3_out, aes(x = pop2, y = est, color = pop2)) +
  geom_point(fill = "red", size=2.5) +
  geom_errorbar(aes(ymin = f3 - se, ymax = f3 + se), width = 0.2) +
  coord_flip() +
  facet_wrap(~ pop3, scales = "free_y") +
  labs(
    title = "Outgroup f3 statistics",
    subtitle = "f3(Outgroup; Wildcats, Domestic cats)",
    x = "Pop2",
    y = "f3 value"
  ) +
  theme_minimal(base_size = 13) +
  theme(
    strip.text = element_text(face = "bold"),
    panel.spacing = unit(1, "lines"), legend.position = "none"
  )

For further interpretation of the results, you can find below a table with the description of each sample (ancient, wild etc).  

<img src="images/ID_labels.png" 
     align="center" 
     width="800" />

<span style="color:red">**Which wildcat is genetically closer to the domestic cats tested? Do you notice any differences in the plots of the domestic cats?**</span>

<div class="alert alert-block alert-success"> 
<b>EXERCISE 1a:</b> Add the wildcats from Sardinia (samples 46_Fsl, 52_Fsl, 53_Fsl, 55_Fsl) to the f3-outgroup analysis. Add new cells below and copy/paste/edit the commands above.
    
- What do you observe?
- Which cats are genetically closer to the wildcats from Sardinia tested? 
- Do you notice any differences in the plots of the four Sardinian wildcats?
</div>

<div class="alert alert-block alert-success"> 
<b>EXERCISE 1b (optional):</b> Add the ancient domestic cats to the analysis (<i>dom_ancient</i>). If needed, adjust the size of the window plots.
    
- What do you observe?
- Do the ancient samples confirm the results? 
</div>

## 5. 𝑓4-statistics in ancient and modern domestic cats
𝑓4 is the covariance of allele frequencies between two pairs of populations. It measures the shared drift in a set of four populations. It can also be expressed as a sum of 𝑓2-statistics, which we have already estimated before. 

By estimating 𝑓4 values, the **treeness** between 4 populations is tested in the form 𝑓4(𝐴,𝐵;𝐶,D) and we are in the presence of an **internal branch**. This means that we are testing whether the relationship between four populations are compatible with a tree described in the initial formula. If they are not, it means that the topology of the tree is wrong or that the treeness is altered by a reticulation (i.e. admixture), and a network better exaplains better the phylogeny tested. If the data are consistent with the tree being tested, then the shared drift between the two pairs of populations (f4-tree value) is equal to zero (there is no overlap between the drift paths of the two pairs, see the figure below).
On the other hand, a value of f4>0 or f4<0 indicates different topologies of the tree (or the presence of gene flow, see later also about the D-statistic).

<center><img src="images/F4_figure.png"
            width="400"/></center>

Let's first recall the populations we are dealing with and the groups we identified before.

In [ ]:
pops

#### 5.1. Genetic affinities with African wildcats form the Levant and Tunisia.
Here we will use 𝑓4-statistics to test different evolutionary models (trees) representing the relationships between domestic cats and African wildcats. We will first test the relationship between modern wildcats from Sardinia and modern and ancient domestic cats against African wildcats (*F. l. lybica*) from the Levant (Israel), and North Africa (Morocco and Tunisia). When an outgroup is included in the test and the 𝑓4 is significantly different from 0, it is possible to redefine the topology of the tree (hence the relationship between the three other populations) based on the sign of the 𝑓4 value. In a tree configuration where the outgroup is in position 4 (pop4), then 𝑓4<0 indicates higher affinity between pop2 and pop3, whereas 𝑓4>0 indicates higher affinity between pop1 and pop3.

<center><img src="images/F4_values.png"
            width="900"/></center>

Here we will use the jungle cat (*F. chaus*) as an outgroup (in position 4), and we will test whether the Sardinian wildcats and domestic cats (in position 3) share more drift with the wildcat from Tunisia (in position 2) and the wildcats from Israel (in position 1). In testing these tree models, we assume that the wild samples are truly wild and did not experience any gene flow with the populations tested. 

In [ ]:
f4 = f4(f2_all, pop1=wc_levant, pop2=wc_tunisia, pop3=c(wc_sardinia,dom_modern), pop4='Felis_chaus') %>% arrange(est)

In [ ]:
library(repr)
options(repr.plot.width=16, repr.plot.height=10)

In [ ]:
f4$Threshold <- cut(f4$z,breaks=c(-Inf,-3,"-3 < qsec <= 3",3,Inf),labels=c("<=-3","-3 < z <= 3",">3"))
f4 %>%
  mutate(pop3 = fct_reorder(pop3, est), sig=as.factor(abs(z)>3)) %>%
  ggplot( aes(x = est, y = pop3,  colour = sig)) +
  geom_point() +
  geom_vline(xintercept = 0, linetype = 2) +
  geom_errorbar(aes(xmin=est-2*se, xmax=est+2*se), width=0.1, size =1, position=position_dodge(0.9)) +
  facet_wrap(~ pop1, scales = "free_y") +
  theme_bw(base_line_size = 0.3, base_rect_size = 1) +
  #facet_wrap(~ pop1, scales = "free_y") +
  labs(
    title = "f4 statistics",
    subtitle = "f4(WC Levant, WC Tunisia, Sardinia/domestic_cat; Felis chaus)",
    x = "modern domestic cat",
    y = "f4 value"
  )  

<span style="color:red">**What is the sign of the 𝑓4 values calculated for the samples tested? How can you interpret the results?**</span>

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If you flip entirely the tree structure in the f4 formula, so f4(Felis chaus; Sardinia/domestic_cat; WC Tunisia, WC Levant), you will notice that the results do not change. On the other hands if you filp only pop3 and pop4, so f4(Felis chaus; Sardinia/domestic_cat; WC Levant, WC Tunisia), the values are the same but with they are positive.
</div>

Let's add now also the ancient domestic cats (*dom_ancient*).

In [ ]:
f4 = f4(f2_all, pop1=wc_levant, pop2=wc_tunisia, pop3=c(wc_sardinia,dom_modern,dom_ancient), pop4='Felis_chaus') %>% arrange(est)


In [ ]:
library(repr)
options(repr.plot.width=16, repr.plot.height=10)

In [ ]:
f4$Threshold <- cut(f4$z,breaks=c(-Inf,-3,"-3 < qsec <= 3",3,Inf),labels=c("<=-3","-3 < z <= 3",">3"))
f4 %>%
  mutate(pop3 = fct_reorder(pop3, est), sig=as.factor(abs(z)>3)) %>%
  ggplot( aes(x = est, y = pop3,  colour = sig)) +
  geom_point() +
  geom_vline(xintercept = 0, linetype = 2) +
  geom_errorbar(aes(xmin=est-2*se, xmax=est+2*se), width=0.1, size =1, position=position_dodge(0.9)) +
  facet_wrap(~ pop1, scales = "free_y") +
  theme_bw(base_line_size = 0.3, base_rect_size = 1) +
  #facet_wrap(~ pop1, scales = "free_y") +
  labs(
    title = "f4 statistics",
    subtitle = "f4(WC Levant, WC Tunisia, Sardinia/domestic_cat; Felis chaus)",
    x = "ancient and modern domestic cat",
    y = "f4 value"
  )  

<span style="color:red">**What do you observe and how can you interpret the results? Do the ancient samples confirm the pattern observed in the modern samples?**</span>

#### 5.2. Genetic affinities with North African wildcats from Tunisia and Morocco.
We will now test with the 𝑓4-statistics whether the modern and ancient domestic cats are more closely related to the North African wildcats from Tunisia or from Morocco. The jungle cat will be used as an outgroup, the populations tested (ancient and modern domestic cats) are in position 3, the wildcat from Tunisia in position 2, the wildcat from Morocco in position 1. 

In [ ]:
f4 = f4(f2_all, pop1=wc_morocco, pop2=wc_tunisia, pop3=c(dom_modern, dom_ancient), pop4='Felis_chaus') %>% arrange(est)

In [ ]:
library(repr)
options(repr.plot.width=10, repr.plot.height=10)

In [ ]:
f4$Threshold <- cut(f4$z,breaks=c(-Inf,-3,"-3 < qsec <= 3",3,Inf),labels=c("<=-3","-3 < z <= 3",">3"))
f4 %>%
  mutate(pop3 = fct_reorder(pop3, est), sig=as.factor(abs(z)>3)) %>%
  ggplot( aes(x = est, y = pop3,  colour = sig)) +
  geom_point() +
  geom_vline(xintercept = 0, linetype = 2) +
  geom_errorbar(aes(xmin=est-2*se, xmax=est+2*se), width=0.1, size =1, position=position_dodge(0.9)) +
  facet_wrap(~ pop1, scales = "free_y") +
  theme_bw(base_line_size = 0.3, base_rect_size = 1) +
  #facet_wrap(~ pop1, scales = "free_y") +
  labs(
    title = "f4 statistics",
    subtitle = "f4(WC Morocco, WC Tunisia, domestic_cat; Felis chaus)",
    x = "ancient and modern domestic cat",
    y = "f4 value"
  )  

<span style="color:red">**What do you observe and how can you interpret the results?**</span>

<div class="alert alert-block alert-success"> 
<b>EXERCISE 2:</b> Add the wildcats from Sardinia (samples 46_Fsl, 52_Fsl, 53_Fsl, 55_Fsl) to the f4 analysis (in position 3). Add new cells below and copy/paste/edit the commands above.
    
- What do you observe?
- How can you explain the pattern observed? 
</div>

## 6. D-statistics and gene flow in ancient and modern European wildcats

The 𝑓4-statistics that we used in the previous section were meant to investigate shared ancestry between phylogenetically close populations, North African wildcats (*Felis lybica lybica*) and domestic cats (*Felis catus*), assuming there was negligible gene flow between them. We will now investigate the ancient and modern European wildcats, which is known to have experienced gene flow with domestic cats. Here, we will test the patterns of gene flow that occurred between European wildcats and both North African wildcats (*Felis lybica lybica*) and domestic cats (*Felis catus*). To do that, we will use D-statistics. 

The differences between 𝑓4-statistics and D-statistics are usually negligible, but D-statistics are formally used to test admixture events and were originally adopted to detect gene flow between humans and Neanderthals ([Green et al. 2010](https://www.science.org/doi/10.1126/science.1188021)). D-statistics take more speficially into account shared derived alleles in the configuration **ABBA-BABA** (the reason why this is also called ABBA-BABA test). The fomula below shows how D-values are computed. The figure is taken from a manuscript by [Malinsky et al. (2020)](https://onlinelibrary.wiley.com/doi/10.1111/1755-0998.13265).

<center><img src="images/D-stats-Malinski.png"
            width="900"/></center>

<center><img src="images/D-stats-formula.png"
            width="300"/></center>

From the formula it appears clear that D values are always between -1 and 1. A positive value indicates an excess of shared derived allele between P1 and P3 (configuration BABA), whereas a negative value indicates an excess of shared derived allele between P2 and P3 (configuration ABBA).

Let's now list all the samples that we are going to analyse in distinct vectors based on geography and age (modern vs ancient). 

In [ ]:
wc_scotland_mod = c('FSX392_Fss','FSX405_Fss')
wc_germany_mod = c('FA699_Fss','FA807_Fss','FB139_Fss','FB434_Fss','FC779_Fss','FD582_Fss','FE384_Fss','FF567_Fss','FF975_Fss','FG711_Fss')
wc_italy_mod = c('1294_Fss','1302_Fss','1611_Fss','66_Fss','78_Fss','79_Fss','997_Fss','99_Fss')
wc_balk_anat_anc = c('ASK01','MENT01','BA02','BA03','KASP01','KASP02','DUR01','DUR02','DUR03','DUR07')
wc_europe_anc = c('BHHL03','CET01','COC01','DOS01','DOS03','ECLY01','MUS01','PRN02','ROC01','ROC02','SAR01')

#### 6.1. Gene flow between ancient European wildcats and African wildcats from the Levant

We will first compute the D-values to test whether there is any signal of gene flow from the African wildcats from the Levant or from domestic cats in the ancient European wildcats (included in two distinct lists, Balkans and the rest of Europe, in pop1). To set up this test, we will use as pop2 an ancient Euopean wildcat that was tested to be "pure" (devoid of any *F. l. lybica/F. catus* ancestry). We will use GLCA01, which originates from Mesolithic Ireland (so earlier than any purported introduction of domestic cats in Europe). In pop3 we will use the sample FSI47_Fsl as representative of the African wildcats from the Levant, and the sample Fcat20822_Ocicat as representative of the domestic cats. 

In [ ]:
D = f4(panel, pop1=c(wc_balk_anat_anc, wc_europe_anc), pop2='GLCA01', pop3=c('FSI47_Fsl','Fcat20822_Ocicat'), pop4='Felis_chaus', f4mode = FALSE) %>% arrange(est)


You must have noticed that we have estimated the f4 directly from the genotype data (stored in the variable *panel* that we created inititally). This operation is slower when dealing with many samples, but it has the advantage that it avoids any problems that may arise from large amounts of missing data. You can read more about it [**here**](https://uqrmaie1.github.io/admixtools/articles/admixtools.html#f4-and-qpdstat). 
We can now plot the D values in two separate plots. 

In [ ]:
# opzioni da impostare per regolare la grandezza della finestra del plot.
library(repr)
options(repr.plot.width=8, repr.plot.height=8)

In [ ]:
D$Threshold <- cut(D$z,breaks=c(-Inf,-3,"-3 < qsec <= 3",3,Inf),labels=c("<=-3","-3 < z <= 3",">3"))
D %>%
  mutate(pop1 = fct_reorder(pop1, est), sig=as.factor(abs(z)>3)) %>%
  ggplot( aes(x = est, y = pop1,  colour = sig)) +
  geom_point() +
  geom_vline(xintercept = 0, linetype = 2) +
  geom_errorbar(aes(xmin=est-2*se, xmax=est+2*se), width=0.1, size =1, position=position_dodge(0.9)) +
  theme_bw(base_line_size = 0.3, base_rect_size = 1) +
  facet_wrap(~ pop3, scales = "free_y") +
  labs(
    title = "D statistics",
    subtitle = "D(ancient Eur wildcats, Mesolithic European wildcat, Levant wildcat/domestic cat; Felis chaus)",
    x = "D value",
    y = "test"
  )  

Interestingly, the results show that most of the ancient wildcats from Anatolia (BA02, BA03, MENT01) and the Balkans or Eastern Europe (DUR samples, KASP01) have significant positive values when testing the gene flow from the Levantine wildcat, but this is not the case when tested against the domestic cat (Ocicat breed). On the contrary, ancient wildcats from Italy and Spain dated to the Neolithic or the Bronze Age (ROC and DOS samples, PRN02, COC01, CET01, SAR01) are not significant in both tests. 

#### 6.2. Including modern wildcats to the analysis
We will now include the modern wildcats from Scotland and plot the results. 

In [ ]:
D = f4(panel, pop1=c(wc_balk_anat_anc, wc_europe_anc, wc_scotland_mod), pop2='GLCA01', pop3=c('FSI47_Fsl','Fcat20822_Ocicat'), pop4='Felis_chaus', f4mode = FALSE) %>% arrange(est)


In [ ]:
# opzioni da impostare per regolare la grandezza della finestra del plot.
library(repr)
options(repr.plot.width=8, repr.plot.height=8)

In [ ]:
D$Threshold <- cut(D$z,breaks=c(-Inf,-3,"-3 < qsec <= 3",3,Inf),labels=c("<=-3","-3 < z <= 3",">3"))
D %>%
  mutate(pop1 = fct_reorder(pop1, est), sig=as.factor(abs(z)>3)) %>%
  ggplot( aes(x = est, y = pop1,  colour = sig)) +
  geom_point() +
  geom_vline(xintercept = 0, linetype = 2) +
  geom_errorbar(aes(xmin=est-2*se, xmax=est+2*se), width=0.1, size =1, position=position_dodge(0.9)) +
  theme_bw(base_line_size = 0.3, base_rect_size = 1) +
  facet_wrap(~ pop3, scales = "free_y") +
  labs(
    title = "D statistics",
    subtitle = "D(ancient-modern Eur wildcats, Mesolithic European wildcat, Levant wildcat/domestic cat; Felis chaus)",
    x = "D value",
    y = "test"
  )

The Scottish wildcats appear to be significant to gene flow when tested against either the Levantine wildcat and the domestic cat. The Scottish wildcat population is a so-called "hybrid swarm", where extensive interbreeding between hybrid individuals and backcrossing with their parent types occur. You can read more about it in [**this study**](https://www.sciencedirect.com/science/article/pii/S0960982223014240).

<div class="alert alert-block alert-success"> 
<b>EXERCISE 3:</b> Add the other modern European wildcats to the analysis (wc_germany_mod, wc_italy_mod). Add new cells below and copy/paste/edit the commands above.
    
- What do you observe?
- How can you explain the pattern observed? 
</div>

We can also plot the results in a single chart to improve the visualization and compare more easily the data. 

In [ ]:
library(forcats)

# Position dodge for consistent shifting (from ggplot)
position_dodge_val <- position_dodge(width = 0.5)

In [ ]:
# Plot
D %>%
  ggplot(aes(x = est, y = fct_inorder(pop1), colour = Threshold, shape = pop3, group = interaction(pop1, pop3))) +
  # Lighter 3*SE error bars
  geom_errorbar(
    aes(xmin = est - 3 * se, xmax = est + 3 * se),
    width = 0.3,
    position = position_dodge_val,
    alpha = 0.3  # Light color for 3*SE
  ) +
  # Standard 1*SE error bars
  geom_errorbar(
    aes(xmin = est - se, xmax = est + se),
    width = 0.3,
    position = position_dodge_val
  ) +
  # Data points with different shapes for pop2
  geom_point(position = position_dodge_val, size = 3) +
  # Reference line at 0
  geom_vline(xintercept = 0, linetype = 2) +
  # Theme and labels
  theme_classic(base_line_size = 0.3, base_rect_size = 1) +
  theme(
    axis.title = element_text(size = 14),     # Axis titles
    axis.text = element_text(size = 12),      # Axis labels
    legend.title = element_text(size = 15),   # Legend titles
    legend.text  = element_text(size = 13)    # Legend labels
  ) +
  ggtitle(
    label = "Dstat gene flow from F. lybica/catus into F. silvestris",
    subtitle = "D(F. chaus, F. lybica/catus; GLCA01, F. silvestris_test)"
  ) +
  xlab("D-value") + 
  ylab("European wildcats") +
  # Clear legend titles
  guides(
    shape = guide_legend(title = "Source (pop2)"),
    colour = guide_legend(title = "Threshold")
  )
